# MBTI to Book Recommendation

In [1]:
import pandas as pd

# Step 1: Load datasets
books = pd.read_csv("books.csv")
book_tags = pd.read_csv("book_tags.csv")
tags = pd.read_csv("tags.csv")

# Step 2: Map tag_id to tag_name
tag_mapping = pd.merge(book_tags, tags, on="tag_id", how="inner")

# Step 3: Define genres based on tag names
genre_keywords = {
    "Science Fiction": ["science-fiction", "sci-fi", "space", "cyberpunk"],
    "Romance": ["romance", "love", "relationship"],
    "Fantasy": ["fantasy", "magic", "dragons", "witch", "wizard"],
    "Mystery": ["mystery", "detective", "crime", "thriller"],
    "Historical Fiction": ["historical", "history", "war"],
    "Horror": ["horror", "ghost", "haunted", "zombie"],
    "Thriller": ["thriller", "suspense", "spy"],
    "Nonfiction": ["nonfiction", "biography", "memoir", "true"],
    "Drama": ["drama", "family", "relationships"],
    "Adventure": ["adventure", "exploration", "journey"],
    "Poetry": ["poetry", "verse", "poem"],
    "Action": ["action", "battle", "fight"],
    "Western": ["western", "cowboy", "wild-west"]
}

def assign_genre(tag_list):
    for genre, keywords in genre_keywords.items():
        if any(keyword in tag_list.lower() for keyword in keywords):
            return genre
    return "General"

# Step 4: Assign genres to books based on tags
tag_mapping["genre"] = tag_mapping["tag_name"].apply(assign_genre)

# Step 5: Merge genres with books
books_with_genres = pd.merge(books, tag_mapping, left_on="book_id", right_on="goodreads_book_id", how="inner")
books_with_genres = books_with_genres[["book_id", "title", "authors", "genre"]].drop_duplicates()

# Step 6: Define MBTI to genre mapping
mbti_to_genre = {
    "INTJ": ["Science Fiction", "Thriller", "Nonfiction"],
    "INTP": ["Science Fiction", "Mystery", "Philosophy"],
    "ENTJ": ["Thriller", "Historical Fiction", "War"],
    "ENTP": ["Fantasy", "Adventure", "Humor"],
    "INFJ": ["Drama", "Fantasy", "Poetry"],
    "INFP": ["Fantasy", "Romance", "Adventure"],
    "ENFJ": ["Historical Fiction", "Drama", "Memoirs"],
    "ENFP": ["Adventure", "Fantasy", "Humor"],
    "ISTJ": ["Historical Fiction", "Mystery", "Crime"],
    "ISFJ": ["Romance", "Drama", "Family"],
    "ESTJ": ["Action", "War", "Crime"],
    "ESFJ": ["Family", "Romance", "Humor"],
    "ISTP": ["Action", "Thriller", "Western"],
    "ISFP": ["Poetry", "Drama", "Romance"],
    "ESTP": ["Action", "Adventure", "Western"],
    "ESFP": ["Comedy", "Adventure", "Romance"]
}

# Step 7: Recommend books based on MBTI
def recommend_books(mbti, books_with_genres, num_recommendations=3):
    genres = mbti_to_genre.get(mbti.upper(), [])
    if not genres:
        return "No genres found for this MBTI type."
    
    # Filter books matching the genres
    recommended_books = books_with_genres[books_with_genres["genre"].isin(genres)]
    if recommended_books.empty:
        return "No books found for the selected genres."
    
    # Select up to `num_recommendations` random books
    recommendations = recommended_books.sample(min(len(recommended_books), num_recommendations))
    
    # Format recommendations as a list
    result = []
    for _, row in recommendations.iterrows():
        result.append(f"Title: {row['title']}\nAuthor: {row['authors']}\nGenre: {row['genre']}")
    return "\n\n".join(result)

# Step 8: Test the recommendation system
user_mbti = "INFP"
print(f"Recommended Books for MBTI ({user_mbti}):\n")
print(recommend_books(user_mbti, books_with_genres, num_recommendations=3))


Recommended Books for MBTI (INFP):

Title: Life, the Universe and Everything (Hitchhiker's Guide, #3)
Author: Douglas Adams
Genre: Adventure

Title: You Had Me At Hello
Author: Mhairi McFarlane
Genre: Romance

Title: The Prophet
Author: Kahlil Gibran
Genre: Romance
